## Classification (b)

In [1]:
from neuralnet import *
from random import shuffle
import numpy as np

# Load the configuration.
config = load_config("./data")

# Create the model
#model  = Neuralnetwork(config)
# Load the data
#x_train, y_train = load_data(path="./data", mode="train") # output normalized and one hot encoding
#x_test, y_test = load_data(path="./data", mode="test")# output normalized and one hot encoding

# Use 10 examples one from each category
'''
# Data loading and spliting
X_b, y_b, X_stats = load_data(path="./data", mode="train")

X_train = []
y_train = []
class_num = list(range(10))


X_train, y_train = [], []
for idx in range(y_b.shape[0]):
    if len(class_num) == 0:
        break
    indice = onehot_decode(y_b[idx])
    
    if indice in class_num:
        X_train.append(X_b[idx])
        y_train.append(y_b[idx])
        class_num.remove(indice)
        
        
X_train = np.array(X_train)        
y_train = np.array(y_train)       
'''

X_train, y_train, X_stats = load_data(path="./data", stats=None, mode="train")

# Get 10 examples, 1 from each category.
X_sub = []
y_sub = []
for k in range(10):
    indices = y_train[:,k] == 1
    X_sub.append(X_train[indices][0])
    y_sub.append(y_train[indices][0])
print(len(X_sub), len(y_sub))
X_sub = np.array(X_sub)
y_sub = np.array(y_sub)

# Load model para
config_prob_b = {}
config_prob_b['layer_specs'] = [3072, 10, 10, 10]
config_prob_b['activation'] = 'tanh'
config_prob_b['learning_rate'] = 0.005
config_prob_b['batch_size'] = 128 
config_prob_b['epochs'] = 100  
config_prob_b['early_stop'] = True 
config_prob_b['early_stop_epoch'] = 5  
config_prob_b['L2_penalty'] = 0  
config_prob_b['momentum'] = False  
config_prob_b['momentum_gamma'] = 0.9  

model_b = Neuralnetwork(config_prob_b)
model_b.forward(X_sub, y_sub)
model_b.backward()




(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
10 10


array([[  6.16256776,  -8.88166446,   4.65124877, ...,  -5.20356103,
          1.73638568,  10.69495567],
       [ 16.8490327 , -33.59014221,   6.97508011, ..., -20.94218458,
         -4.15596622,  29.36727404],
       [ -3.59779998,  -0.10507974,  -1.94844756, ...,  15.13638649,
         -0.88855443, -16.11482895],
       ...,
       [ -7.91144485,  13.48843793,  -4.52109209, ...,   9.44241916,
         -4.96564945, -17.51147168],
       [  4.99169585, -10.6426176 ,  21.1392499 , ...,   4.1516606 ,
          6.86621007, -12.68241444],
       [ -9.03211193,  12.12942032,   1.60684442, ...,   6.27620176,
         13.28047936,  -6.25968368]])

In [2]:
# d_b d_w comparison

## Part (b) Estimation of bias weight and weight
def Num_Est_b(model, layer, eps, output_idx):
    layer.b[0][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_sub, y_sub)[1]
    print(loss_1)
    layer.b[0][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_sub, y_sub)[1]
    print(loss_2)
    layer.b[0][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

def Num_Est_w(model, layer, eps, input_idx, output_idx):
    layer.w[input_idx][output_idx] += eps # E(w+e)
    loss_1 = model.forward(X_sub, y_sub)[1]
    layer.w[input_idx][output_idx] -= 2*eps # E(w-e)
    loss_2 = model.forward(X_sub, y_sub)[1]
    layer.w[input_idx][output_idx] += eps # back to normal
    return (loss_1 - loss_2) / (2 * eps) # Numerical estimation for dEdW

print("Layers:", len(model_b.layers))
# Weights to modify:
# 1 output bias weight
# 1 hidden bias weight for each hidden layer
# 2 hidden to output weights
# 2 input to hidden weights
# Show that the grad between is within O(eps^2) of backprop weights.
db = []
dw = []
db_est = []
dw_est = []

eps = 10e-2
# Hidden Layer 1
hidden_layer_1 = model_b.layers[0]
# Hidden Layer 2
hidden_layer_2 = model_b.layers[2]
# Output Layer
output_layer = model_b.layers[4]

# 1 output bias weight
input_idx = 0
output_idx = 1
cur_layer = output_layer
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Output bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 1 hidden bias weight for hidden layer 2
cur_layer = hidden_layer_2
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Hidden Layer 2 bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 1 hidden bias weight for hidden layer 1
cur_layer = hidden_layer_1
db_est.append(Num_Est_b(model_b, cur_layer, eps, output_idx))
db.append(cur_layer.d_b[0][output_idx]) 
print(f"Hidden Layer 1 bias weight diff: {db_est[-1] - db[-1]:.6f}")

# 2 hidden to output weights
input_idx = 0
output_idx = 1
cur_layer = hidden_layer_2
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Hidden to output weight diff 1: {dw_est[-1] - dw[-1]:.6f}")
input_idx = 0
output_idx = 2
dw_est.append(Num_Est_w(model_b, cur_layer, eps, input_idx, output_idx))
dw.append(cur_layer.d_w[input_idx][output_idx])
print(f"Hidden to output weight diff 2: {dw_est[-1] - dw[-1]:.6f}")



Layers: 5
2.7955808287458708
2.804619725137625
Output bias weight diff: 0.000050
2.81047141362105
2.787648238977348
Hidden Layer 2 bias weight diff: -0.003901
2.7998624079701417
2.7998825342605023
Hidden Layer 2 bias weight diff: 1.890356
Hidden to output weight diff 1: 0.596210
Hidden to output weight diff 2: -0.160733
